# Modular Gaussian Integers

*Dmytro Fedoriaka, July 2024*

## 1. Introduction

In competitive programming it is common to do computations modulo a prime number $M$. It is convenient because one can define and efficiently compute an inverse modulo prime number (e.g. using formula $x^{-1} \equiv x^{M-2} (\text{mod} ~M))$. In other words, $\mathbb{Z}_M$ is a field. So, one can do arithmetic pretty much like if these numbers are real numbers and get correct answers.

However, there are other operations that are not well-defined in a field $\mathbb{Z}_M$. In particular, a square root is not defined for all elements in $\mathbb{Z}_M$. For example, there is no solutions to equation $x^2 = 3$ in $\mathbb{Z}_7$, which means square roots of 3 is not defined.

What to do in this case when we still need to get square root for any number? Well, what we do for real numbers $\mathbb{R}$ when we want to extract square root of a negative number? We define a new symbol $i$ and say that by definition $i^2=1$, then add it to $\mathbb{R}$ and also close new structure under all arithmetic operations (this construction is called a [field extension](https://en.wikipedia.org/wiki/Field_extension)). As a result we get new field $\mathbb{R}[i]=\mathbb{C}$, which we call "complex numbers".

Here I propose to do the same thing to a field of residuals modulo M. It turns out that resulting construction has some interesting properties. It is a field and it allows to extract square roots of any number from $\mathbb{Z}_M$, and much more.

Below I will describe some properties of $\mathbb{Z}_M$ and describe how to efficiently (in $O(poly(\log M))$) compute roots of any degree in it, and order of any non-zero element. Finally, I will show a practical application of this constructions: solving cubic equations in $\mathbb{Z}_M$.

**Note.** *I couldn't find literature explicitly describing properties $\mathbb{Z}_M[i]$, but it's hard to believe it doesn't exist. So I didn't bother to give rigorous mathematical proofs (so far). Instead, I focused on describing interesting facts and providing explicit algorithms in Python. Also, my notation might be different from conventional.*

## 2. Definition

Let $M$ be a prime number such that $M \equiv 3 ~ (\text{mod} ~ 4)$.

Define $\mathbb{Z}_M$ - a field of residuals modulo $M$.

Define $\mathbb{Z}_M[i] = \big \{ a+b \cdot i ~|~ a,b \in \mathbb{Z}_M \big \}$, where $i$ is such a symbol that by definition $i^2 = 1$. 

I call these "Modular Gaussian Integers", because these are like [Gaussian Integers](https://en.wikipedia.org/wiki/Gaussian_integer) $\mathbb{Z}[i]$, except their coefficients are from $\mathbb{Z}_M$ instead of $\mathbb{Z}$.

## 3. Elementary operations


**Arithmetic operations** are defined the same way as for regular complex numbers $\mathbb{C} = \mathbb{R}[i]$. Note that coefficients are from $\mathbb{Z}_M$, so it's assumed that all operations with coefficients are modulo M (e.g. when we write $ab$, we imply $(ab)\%M$) .

* Addition: $(a+b i) + (c+di) = (a+c) + (b+d)i$
* Subtraction: $(a+b i) - (c+di) = (a+c) - (b+d)i$
* Multiplication: $(a+b i) \cdot (c+di) = (ac-bdc) - (ad+bc)i$
* Division: $\frac{a+bi}{c+di} = \frac{(a+bi)(c-di)}{(c+di)(c-di)} = \frac{(ac+bd) + (bc-ad)i}{c^2+d^2} = 
(ac+bd)\cdot (c^2+d^2)^{-1} + (bc-ad) \cdot (c^2+d^2)^{-1} \cdot i $

Division involves finding inverse of $(c^2+d^2)$ modulo $M$. So, $\mathbb{Z}_M[i]$ is a field if and only if this inverse exists for any non-zero $c+d i$. This holds if and only if $M$ is prime and $M \equiv 3 ~(\text{mod}~ 4)$. See proof [here](https://math.stackexchange.com/questions/2160330/about-ring-of-gaussian-integers-modulo-n).

Formally, we can claim that:
* $\mathbb{Z}_M$ is a ring for any integer $M \ge 2$.
* $\mathbb{Z}_M$ is a field if and only if $M$ is prime and $M \equiv 3 ~(\text{mod}~ 4)$.

**Exponentiation.** We can define $a^b$ as multiplying $a$ by itself $b$ times. This can be computed in $O(\log M)$ using binary exponentiation.

**Complex conjugate.** Define $\overline{a+b i}= a-bi$.

**Modulus.** Define $|a+b i| = a^2+b^2 \in Z_M$. Note that unlike in $\mathbb{C}$, we **are not taking square root**, because it's not always possible in $Z_M$. 

It is easy to check that:
  * $|x|=x \cdot \overline{x}$
  * $|x| \cdot |y| = |x \cdot y|$
  * $|x|=0 \Leftrightarrow x=0$
  * $|x/\overline{x}|=|\overline{x}/x|=1$ for any non-zero $x$.

## 4. Structure of multiplicative group of $\mathbb{Z}_M[i]$

*This section is not very useful for algorithms and can be skipped.*

We know that in $\mathbb{C}$ any number can be written as $z = A \cdot e^{i \phi}$, where $A \in \mathbb{R}, A\ge 0$ and $\phi \in [0, 2\cdot \pi)$. Let's do something similar in $Z_M[i]$. In $\mathbb{C}$ we could just use $A=|z|$, $e^{i \phi} = z/|z|$. here it's a bit more complicated.

**Lemma.** Any non-zero $x = \mathbb{Z}_M[i]$ can be written as $x \in A \cdot B$,
where are $A \in Z_M$ (i.e. it's "real") and $|B|=\pm 1$.

**Proof.**
Consider $|x|$. There are two possible cases
   
   1. $|x|$ has square roots $\pm r_1$ with $r_1 \in \mathbb{Z}_M, r_1 \ne 0$, Then $r1_2=|x|$. Take $A=r_1, B=x/r_1$. Then $A \in \mathbb{Z}_M, |B| = \frac{|x|}{r_1^2}=1$. 
   
   2. $|x|$ has square roots $\pm r_2 \cdot i$ with $r_2 \in \mathbb{Z}_M, r_2 \ne 0$. Then $r2_2=-|x|$. Take $A=r_2, B = x/r_2$. Then $A \in \mathbb{Z}_M, |B| = \frac{|x|}{r_2^2}=-1$. 


Next, note that set of all numbers with modulus $\pm 1$ forms a group under multiplication of order $2(M+1)$. Denote $\omega_2$ - a primitive root in it (an element with order $2(M+1))$. Then we can write our representation as

$$ x = A \cdot \omega_2 ^{n_2},$$

where $n_2 \in [0, 2(M+1))$.

All real numbers also form a subgroup, which has a primitive root $\omega_1$. So, we can write 

$$ x = \omega_1 ^{n_1} \cdot \omega_2 ^{n_2},$$

where $n_1 \in [0, M-1),  n_2 \in [0, 2(M+1))$.

This representation is non-unique. In fact, every $x$ has exactly two representations of this form, because 

$$  \omega_1 ^{n_1} \cdot \omega_2 ^{n_2} =  (\omega_1 ^{n_1} \cdot (-1)) \cdot (\omega_2 ^{n_2} \cdot (-1)) = 
\omega_1 ^{(n_1+(M-1)/2)\%(M-1)} \cdot \omega_2 ^{(n_2+(M+1))\%(2M+2)} $$

We can break ambiguity by restricting $n_1$ to $[0, (M-1)/2)$. Define $K=(M-1)/2$. Then $2(M+1)=4(K+1)$. Recall that $M\%4=3$, so $K$ is odd, then $gcd(K,4(K+1))=1$. We can write $x = \omega_1 ^{n_1} \cdot \omega_2 ^{n_2}$, where $n_1 \in [0, K), n_2 \in [0,4(K+1)$. Using Chinese Remainder Theorem, now we can claim that:

**Lemma.** Any non-zero $x \in \mathbb{Z}_M[i]$ can be written in unique way:

* as $x = \omega_1 ^{n_1} \cdot \omega_2 ^{n_2}$, where $n_1 \in [0, K), n_2 \in [0,4(K+1)$,
* as $x = (\omega_1 \omega_2) ^{n}$, where $n \in [0, M^2-1)$.

So, we have proven that multiplicative group of $Z_M[i]$ is cyclic with generator $\omega = \omega_1 \cdot \omega_2$.


How to efficiently find $\omega_1$ and $\omega_2$?
* To find $\omega_1$, randomly generate an integer $a \in [2, M-1]$, until $ord(a + 0 \cdot i)=M-1$.
* To find $\omega_2$, randomly generate an integer $a \in [1, M-1]$. Consider $b = \sqrt{-1-a^2}$. If it's not real. Otherwise, check if $ord(a + b \cdot i)=2M+1$.

In both cases we will need to do only $O(1)$ attempts.

Of course, we didn't need to go to all this trouble, as it is a known fact that that multiplicative group of a finite field is a cyclic group. However, this representation will be useful for taking discrete logarithms.



Some other interesting facts about the multiplicative group:


* Of $M-1$ non-zero elements in $Z_M$, exactly half have roots in $Z_M$.

* For any two non-zero numbers $x$ and $-x$ from $Z_M$, one of them has two real roots $\pm y$, and the other has two purely imaginary roots $\pm y \cdot i$. 
    * **Proof sketch.** Consider mapping $f: Z_M \to Z_M$, $f(x)=x^2$. Obviously, it maps numbers $x$ and $-x$
       to the same number. But also if it maps some number $x$ to $y$, it doesn't map any number to $-y$ (in
       other words, there are no such $a,b \ne 0$ that $a^2=-b^2$ because that would mean $a^2+b^2=0$).
* Of $M^2-1$ non-zero elements in $Z_M[i]$, exactly half have roots in $Z_M$ (again, each of them having two roots summing to 0), and the other half don't.


## 5. Computing order in $\mathbb{Z}_M[i]$ in $O(\log(M)^2)$

**Definition.** Order of non-zero $x \in \mathbb{Z}_M[i]$ is a smallest positive integer $n$ such that $x^n=1$.

By the [Lagrange theorem](https://en.wikipedia.org/wiki/Lagrange%27s_theorem_(group_theory)), the order of $x$ is divisor of the size of group, i.e. divisor of $(M-1)(M+1)$, so we can just check all divisors $d$ in order and find the smallest of them such that $x^d=1$. However, we can do much faster.

Let's say that if $x^d=1$, then $d$ is *candidate order*. Consider known prime factorization $M^2-1 = p_1^{a_1}p_2^{a_2}...p_l^{a_l}$. We know that the answer will be 
$p_1^{b_1}p_2^{b_2}...p_l^{b_l}$ where $0 \le b_i \le a_i$. Also we know that if $X$ is a candidate order, any its multiple is candidate order, and if $X$ is not candidate order, any divisor of $X$ is not a candidate order.

Then we can propose the following algorithm:
* Start with $A = p_1^{a_1}p_2^{a_2}...p_l^{a_l}$.
* For all $i=1 \dots l$:
   * While $a_i > 0$ and $x^A = 1$ decrement $a_i$.
   * If $x^A \ne 1$, increment $a_i$.
* Resulting X is the order of $x$.

This algorithms needs to compute power at most $a_1+a_2 +\cdot + a_l = O(\log(M))$ times, and computing power is done in $O(\log(M))$, so the total complexity is $O(\log(M^2)).$


**Note about complexity.** Here and below we assume that we already have factorization of $M^2-1$, and don't include cost of computing this factorization into complexity of operations. In practice, for $M$ up to $10^9$ we can use naive factorization of $M^2-1=(M-1)(M+1)$ by checking all divisors up to root, having compliexity $O(\sqrt{M})$. For larger $M$ we could use more advanced factoring algorithms. 

## 6. Computing $r^{th}$ roots in $\mathbb{Z}_M[i]$ in $O(\log^4(M) + r \log^3(M))$

There is a classic [Tonelli-Shanks](https://en.wikipedia.org/wiki/Tonelli%E2%80%93Shanks_algorithm) algorithm that computes square roots in $Z_M$. Actually, it can be generalized to compute any root in any finite field. This is called Adleman-Manders-Miller algorithm, and it is well-described in [this paper](https://arxiv.org/pdf/1111.4877) by Zhengjun Cao, Qian Sha and Xiao Fan (table 4 on page 8), so I won't repeat it here.

Some implementation details:
* $q = M^2$
* When it says, "Compute the least nonnegative integer α such that s|rα − 1.", this just means $\alpha = r^{-1} (\text{mod} ~s)$.
* When it says to compute $\log_a d$, just find smallest $j$ such that $a^j=d$ by checking $j=0,1,\dots,r-1$, and it's guaranteed that you'll find the answer.
* It covers only cases when $r$ is prime and divisor of $M^2-1$.
  * If $r$ is composite, just compose roots for all prime factors, e.g. $\sqrt[6]{x} = \sqrt[2]{\sqrt[3]{x}}.$
  * If $r$ is prime but not a divisor of $M^2-1$, then $\sqrt[r]{x}=x^{r^{-1}(\text{mod}~M^2-1)}$.
  
The complexity $O(\log^4(M) + r \log^3(M))$ is from the same paper.

Now when we found one root, how to find all roots? Just multiply the found root by all $r$-th roots of unity. The $r$-th roots of unity are easy to compute using the fact that multiplicative group is cyclic group. There will be $g=GCD(r, M^2-1)$ of them, and they will be of form $\omega^{kj}$ where $k=\frac{M^2-1}{g}$ and $j<=0,1,\dots,g-1$.

## 7. Computing discrete logarithm in $\mathbb{Z}_M[i]$ in $O(\sqrt{M})$ (sketch)

Let's say that $\log_A{x}=y$ if $A^y=x$. How to efficiently find $\log_A{x}$.

Decompose $x=\omega_1^{x_1} \omega_2^{x_2}$, $A=\omega_1^{A} \omega_2^{A}$. Then we are looking for such an integer $y$ that

$$\omega_1^{A_1 y} \omega_2^{A_2 y} = \omega_1^{x_1} \omega_2^{x_2}
\Rightarrow
\begin{cases}
    A_1 y \equiv x_1 (\text{mod} ~M-1) \\
    A_2 y \equiv x_2 (\text{mod}~ 2(M+1))
\end{cases}
$$

This can be solved using Chinese Remainder Theorem and the [Extended Euclidean Algorithm](https://cp-algorithms.com/algebra/extended-euclid-algorithm.html). Note that answer doesn't always exist.

But how to compute the decompositions? Above we have shown how to compute decomposition $x=AB$ where $A=\omega_1^{n_1}$, $B=\omega_2^{n_2}$ but not how to find $n_1,n_2$. Now we have the problem: given $x$ in multiplicative subgroup of order $s$ and primitive root $\omega$ of that subgroup, find $n$ such that $x=\omega^x$. This can be solved using the [Baby-step giant-step algorithm](https://cp-algorithms.com/algebra/discrete-log.html) in $O(s)$. So the overall complexity for the original task is $O(\sqrt{M})$.

Note that we could apply the BSGS algorithm directly in $\mathbb{Z}_M[i]$ and that would have complexity $O(\sqrt{M^2-1}) = O(M)$. We reduced this complexity by factoring the group. This makes me think that we can continue this and factor the multiplicative group into as many "orthogonal" subgroups as many there are factors in prime decompositions of $M^2-1$ and then complexity of discrete log would be $O(\sqrt{p})$, where $p$ is the largest prime factor in $M^2-1$. However, in the worst case it's still $O(\sqrt{M})$.

## 8. Implementation

Below is the full implementation of the Modular Gaussian Integers in Python.

Notes:
* It's deliberately written with no dependencies on external libraries, so it could be used in competitive programming.
* Prime factoring is done using naive algorithm, which is fine for $M \le 10^9$. The factoring needs to be only computed once for given $M$ and the is cached. To work with larger $M$, use `sympy.primefactors` from SymPy instead of `primefactors` (although this code will still work for any $M$, just be slower).
* If rewriting in C++, for $M>2^{31}$ you will need long arithmetic.

In [1]:
import functools
import math
import random


@functools.cache
def primefactors(x: int) -> set[int]:
    ans = set()
    for d in range(2, x):
        if d*d > x:
            break
        while x % d == 0:
            ans.add(d)
            x //= d
    if x > 1:
        ans.add(x)
    return ans
  
is_prime = lambda x: primefactors(x) == {x}
  
def pow_mod(x: int, y: int, M: int) -> int:
    """Returns (x**y)%M."""
    ans = 1
    while y != 0:
        if (y & 1):
            ans = (ans*x) % M
        x = (x*x) % M
        y >>= 1
    return ans


class ModGaussianPool:
    """Manages all Modular Gaussian numbers with the same modulo M."""

    def __init__(self, M: int):
        assert is_prime(M), "M is not prime."
        assert M % 4 == 3
        self.M = M

        # Prime factors of M^2-1 (order or multiplicative group).
        self.M2m1_prime_factors = primefactors(M-1).union(primefactors(M+1))

        # Find primitive root in subgroup of real numbers.
        for i in range(2, M):
            if self.num(i).get_order() == M-1:
                self.w1 = self.num(i)
                break
        assert (self.w1.pow(self.M - 1).is_one())
        assert (self.w1.pow((self.M - 1)//2) == self.num(-1))

        # Find primitive root in subgroup of elements having modulus +1/-1.
        while True:
            a = random.randint(1, self.M-1)
            b2 = self.num(-1-a**2)
            b = b2._discrete_root_amm(2)
            if b.b != 0:
                continue
            x = self.num(a, b.a)
            assert (x.a**2+x.b**2) % M == M - 1
            if x.get_order() == 2*(self.M + 1):
                self.w2 = x
                break
        assert self.w2.pow(2*(self.M + 1)).is_one()
        assert self.w2.pow(self.M + 1) == self.num(-1)
        assert (self.w1*self.w2).get_order() == self.M**2 - 1

    def __repr__(self):
        return "ModGaussianPool(%d)" % self.M

    @functools.cache
    @staticmethod
    def make_pool(M: int):
        return ModGaussianPool(M)

    @functools.cache
    def unity_roots(self, r: int):
        n = self.M**2-1
        gen = self.w1*self.w2
        return [gen.pow(i) for i in range(0, n, n // math.gcd(r, n))]
      
    @functools.cache
    def totient(self, x: int) -> int:
        assert (self.M**2 - 1) % x == 0
        for p in self.M2m1_prime_factors:
            if x % p == 0:
                x = (p-1)*(x//p)
        return x
    
    def inv_mod(self, x: int, s: int) -> int:
        """Returns x^-1 (mod s), where s is divisor of M^2-1."""
        assert math.gcd(x, s) == 1, "Not invertible"
        return pow_mod(x, self.totient(s)-1, s)  

    def all_numbers(self):
        return (ModGaussian(i, j, self) for i in range(self.M) for j in range(self.M))

    def num(self, a: int, b: int = 0) -> "ModGaussian":
        return ModGaussian(a, b, self)

    def random(self) -> "ModGaussian":
        return ModGaussian(random.randint(0, self.M-1), random.randint(0, self.M-1), self)

    @functools.cache
    def get_rho(self, r: int) -> "ModGaussian":
        """Returns non-zero element that is not an r-th root."""
        p = (self.M**2-1)//r
        while True:
            rho = self.random()
            if not (rho.is_zero() or rho.pow(p).is_one()):
                return rho


class ModGaussian:
    def __init__(self, a: int, b: int, pool: ModGaussianPool):
        self.a = a % pool.M
        self.b = b % pool.M
        self.pool = pool

    def __add__(self, other):
        assert self.pool is other.pool
        return ModGaussian(self.a+other.a, self.b+other.b, self.pool)

    def __sub__(self, other):
        assert self.pool is other.pool
        return ModGaussian(self.a-other.a, self.b-other.b, self.pool)

    def __mul__(self, other):
        if type(other) is int:
            return ModGaussian(self.a*other, self.b*other, self.pool)
        assert self.pool is other.pool
        return ModGaussian(self.a*other.a - self.b*other.b, self.a*other.b + self.b*other.a, self.pool)

    def __truediv__(self, other):
        if type(other) is int:
            assert other != 0, "Division by zero"
            inv = pow_mod(other, self.pool.M-2, self.pool.M)
            return ModGaussian(self.a*inv, self.b*inv, self.pool)
        assert self.pool is other.pool
        a = self.a*other.a + self.b*other.b
        b = self.b*other.a - self.a*other.b
        denom = (other.a**2 + other.b**2) % self.pool.M
        assert denom != 0, "Division by zero"
        denom_inv = pow_mod(denom, self.pool.M-2, self.pool.M)
        return ModGaussian(a * denom_inv, b * denom_inv, self.pool)

    def __pow__(self, other):
        return self.pow(other)

    def is_zero(self):
        return self.a == 0 and self.b == 0

    def is_one(self):
        return self.a == 1 and self.b == 0

    def __repr__(self):
        return str(self.a) if self.b == 0 else "%d+%dj" % (self.a, self.b)

    def __eq__(self, other):
        if type(other) is not ModGaussian:
            return False
        return self.a == other.a and self.b == other.b and self.pool is other.pool

    def __hash__(self):
        return hash(self.a * self.pool.M + self.b)

    def pow(self, y):
        ans = ModGaussian(1, 0, self.pool)
        x = self
        y %= (self.pool.M**2 - 1)
        while y != 0:
            if (y & 1):
                ans = ans*x
            x = x*x
            y >>= 1
        return ans

    def conj(self):
        return ModGaussian(self.a, -self.b, self.pool)

    def get_order(self) -> int:
        """Returns smallest i, such that self^i=1."""
        assert not self.is_zero(), "Order is not defined for the zero element."
        ans = self.pool.M**2 - 1
        assert self.pow(ans).is_one()
        for f in self.pool.M2m1_prime_factors:
            while ans % f == 0:
                ans //= f
                if not self.pow(ans).is_one():
                    ans *= f
                    break
        return ans

    # Finds disrete root for the case when r is prime.
    # Reports None if root doesn't exist.
    # Uses Adleman-Manders-Miller algorithm, see https://arxiv.org/pdf/1111.4877
    def _discrete_root_amm(self, r) -> "Optional[ModGaussian]":
        # Handle trivial case when r is not a divisor of M^2-1.
        qm1 = self.pool.M**2-1  # "q-1" in the paper.
        if qm1 % r != 0:
            return self.pow(self.pool.inv_mod(r, qm1))

        # Check if it's a r-th residue.
        if not self.pow(qm1//r).is_one():
            return None

        rho = self.pool.get_rho(r)
        s, t = qm1, 0
        while s % r == 0:
            s //= r
            t += 1
        assert (t >= 1)

        al = self.pool.inv_mod(r, s)  # least non-negative such that (r*al-1)%s==0
        assert ((r*al-1) % s == 0)

        a = rho.pow(r**(t-1)*s)
        assert not a.is_zero()
        b = self.pow(r*al-1)
        c = rho.pow(s)
        h = self.pool.num(1)
        for i in range(1, t):
            d = b.pow(r**(t-1-i))

            # Compute the discrete logarithm.
            a_j = self.pool.num(1)
            j = None
            for j1 in range(r):
                if d == a_j:
                    j = -j1
                    break
                a_j = a_j * a
            assert j is not None

            b = b * c.pow(r).pow(j)
            h = h * c.pow(j)
            c = c.pow(r)
        return self.pow(al) * h

    def discrete_root(self, r) -> "Optional[ModGaussian]":
        """Finds some r-th root or reports that there is none."""
        if self.is_zero():
            return self
        ans = self
        for f in primefactors(r):
            while r % f == 0:
                r /= f
                ans = ans._discrete_root_amm(f)
                if ans is None:
                    return None
        return ans

    def all_roots(self, r) -> "List[ModGaussian]":
        """Returns all r-th roots."""
        if self.is_zero():
            return [self]
        r %= (self.pool.M**2 - 1)
        r0 = self.discrete_root(r)
        if r0 is None:
            return []
        return [r0 * ur for ur in self.pool.unity_roots(r)]

### Example usage

In [2]:
pool = ModGaussianPool.make_pool(19)
a = pool.num(1,14)
b = pool.num(3,4)
print(f"a={a}, b={b}, a+b={a+b}, a-b={a+b}, a*b={a*b}, a/b={a/b}")
print(f"a**5={a**5}")
print(f"order(a)={a.get_order()}")
print(f"Square roots of a: {a.all_roots(2)}")
print(f"Cubic roots of a: {a.all_roots(3)}")

a=1+14j, b=3+4j, a+b=4+18j, a-b=4+18j, a*b=4+8j, a/b=13
a**5=7
order(a)=15
Square roots of a: [14+10j, 5+9j]
Cubic roots of a: [10+1j, 15+11j, 13+7j]


### Tests

In [3]:
# Exhaustive check for orders.
def order_bf(x):
  assert not x.is_zero()
  ans = 1
  a = x
  while not a.is_one():
    a = a * x
    ans += 1
  return ans

for M in [3, 7, 11, 19, 23, 31]:
  pool = ModGaussianPool.make_pool(M)
  for x in pool.all_numbers():
    if x.is_zero(): continue
    assert order_bf(x) == x.get_order()
  print("Exhaustive order check for M=%d OK" % pool.M)

Exhaustive order check for M=3 OK
Exhaustive order check for M=7 OK
Exhaustive order check for M=11 OK
Exhaustive order check for M=19 OK
Exhaustive order check for M=23 OK
Exhaustive order check for M=31 OK


In [4]:
for M in [3,7,11,19,23, 31]:
    pool = ModGaussianPool.make_pool(M)
    roots_bruteforce = dict()
    for i in range(2, 6):
        roots_bruteforce[i] = {x:[] for x in pool.all_numbers()}
        for x in pool.all_numbers():
            roots_bruteforce[i][x.pow(i)].append(x)

    for x in pool.all_numbers():
        for i in range(2, 6):
            assert set(roots_bruteforce[i][x]) == set(x.all_roots(i))

    print("Exhaustive root check for M=%d OK" % pool.M)
  


Exhaustive root check for M=3 OK
Exhaustive root check for M=7 OK
Exhaustive root check for M=11 OK
Exhaustive root check for M=19 OK
Exhaustive root check for M=23 OK
Exhaustive root check for M=31 OK


In [5]:
for M in [103, 271, 991, 10**9+7]:
  pool = ModGaussianPool.make_pool(M)
  for _ in range(1000):
      x = pool.random()
      r = random.randint(2, 10)
      roots = x.all_roots(r)
      for root in roots:
          assert root**r == x
  print("Random root check for M=%d OK" % pool.M)

Random root check for M=103 OK
Random root check for M=271 OK
Random root check for M=991 OK
Random root check for M=1000000007 OK


In [6]:
for M in [3,7,11,19,23]:
    pool = ModGaussianPool.make_pool(M)
    non_zeros = set(x for x in pool.all_numbers() if not x.is_zero())
    # Verifying that multiplicative group is cyclic group.
    gen = pool.w1 * pool.w2
    assert set(gen.pow(i) for i in range(M**2-1)) == non_zeros
    
    # Verifying the unique representation.
    s1 = set()
    for i1 in range((pool.M-1)//2):
        for i2 in range(2*(pool.M+1)):
            v1 = pool.w1.pow(i1)
            v2 = pool.w2.pow(i2)
            s1.add(v1 * v2)
    assert s1 == non_zeros
            
    # Verifying number of primitve roots (aka generators).
    pr_count = 0
    for x in pool.all_numbers():
        if x.is_zero(): continue
        if x.get_order() == pool.M**2-1: pr_count += 1
    assert pr_count/(pool.M**2-1) >= 0.25    
    print("Exhaustive structure check for M=%d OK" % pool.M)


Exhaustive structure check for M=3 OK
Exhaustive structure check for M=7 OK
Exhaustive structure check for M=11 OK
Exhaustive structure check for M=19 OK
Exhaustive structure check for M=23 OK


## 9. Application: solving cubic equations in $\mathbb{Z}_M$

Consider the following problem: find all integer $x \in [0, M)$, such that

$$ax^3+bx^2+cx+d \equiv 0 ~(\text{mod} ~M).$$

After reducing it to "depressed cubic" $t^3+pt+q=0$ and considering trivial case $p=0$, we can solve it using the [Cardano's formula](https://en.wikipedia.org/wiki/Cubic_equation#Cardano's_formula):

$$t = C-\frac{p}{3C}, \text{where} ~C = \sqrt[\leftroot{-1}\uproot{2}\scriptstyle 3]{-\frac{q}{2}+\sqrt{\frac{q^2}{4}+\frac{p^3}{27}}}$$

When coefficients are from $\mathbb{R}$, this formula can be used as follows: take any of 2 square roots, then then take all 3 cubic roots, and then keep those that are real. However, it is well-known that even if all solutions are real, intermediary results might be complex, and there is no alternative way to compute this without going into $\mathbb{R}$.

The same works for $\mathbb{Z}_M$. In order to solve the equation where coefficients and answers are $\mathbb{Z}_M$, we will have to go into $\mathbb{Z}_M[i]$ for intermediary calculations.

To be concrete, for in $\mathbb{Z}_{11}$, the equation $x^3+x+2$ has 3 roots: 5,7 and 10. However, when solving it using Cardano's formula, we will need to compute square root from 10, which doesn't exist in $\mathbb{Z}_{11}$.

Below is code to solve cubic equation in $\mathbb{Z}_M$ for case $a \ne 0$ and tests.

In [7]:
def solve_cubic_eqn(a: int, b: int, c: int, d: int, M: int) -> set[int]:
    """Solves a*x^3+b*x^2+c*x+d=0 in Z[M] using the Cardano's formula."""
    assert M>3  # Doesn't work for case M=3, but it can be trivially solved with brute force.
    pool = ModGaussianPool.make_pool(M)
    a, b, c, d = map(lambda x: pool.num(x), [a, b, c, d])
    assert not a.is_zero()
  
    # Reduce this to the "Depressed cubic" t^3+pt+q=0.
    p = (a*c*3-b*b)/(a*a*3)
    q = (b*b*b*2 - a*b*c*9+a*a*d*27)/(a*a*a*27)

    ts=[]
    if p.is_zero():
        ts = (q*-1).all_roots(3)
    else:
        r = ((q*q/4)+(p*p*p/27)).discrete_root(2)        
        ts = [C - p/(C*3) for C in (q/(-2) + r).all_roots(3)]
        
    xs = set(t-b/(a*3) for t in ts)
    return {x.a for x in xs if x.b==0}

In [8]:
import itertools
  
def solve_cubic_bf(a: int, b: int, c: int, d: int, M: int) -> set[int]:
    return {x for x in range(M) if (a*x*x*x+b*x*x+c*x+d)%M==0}  
  
for M in [7, 11]:
    for a,b,c,d in itertools.product(range(1,M), range(M), range(M), range(M)):
        bf_sol = solve_cubic_bf(a,b,c,d,M)
        sol = solve_cubic_eqn(a,b,c,d,M)
        assert sol == bf_sol
    print("Exhaustive OK for %d" % M)

for M in [19, 23, 31, 43, 103, 271, 991]:
    for _ in range(1000):
        a=random.randint(1,M-1)
        b=random.randint(0,M-1)
        c=random.randint(0,M-1)
        d=random.randint(0,M-1)
        bf_sol = solve_cubic_bf(a,b,c,d,M)
        sol = solve_cubic_eqn(a,b,c,d,M)
        assert sol == bf_sol
    print("Random OK for %d" % M)

Exhaustive OK for 7
Exhaustive OK for 11
Random OK for 19
Random OK for 23
Random OK for 31
Random OK for 43
Random OK for 103
Random OK for 271
Random OK for 991


## 10. Open questions

*I'm pretty sure these are answered in some book or paper, but at the moment I don't know the answers.* 

**1. Further field extension.**

What if we wanted to be able  to compute square root for **any** element in $\mathbb{Z}_M[i]$? For this we need to go to further extension $\mathbb{Z}_M[i,j]$, where $j$ is a symbol defined such that $j^2 = \omega_2$. Will this be a field? How many elements are in this field (I think $M^4$)? Will it be isomorphic to field of quaternions over $\mathbb{Z}_M$?

**2. Infinite field extension.**

What if we added ALL roots of 1 of any degree? Formally, for any integer $r$ we add $r$-th root of unity if it's not already there (obviously result will have infinite size)? Is this a field? Is it described anywhere?

**3. Fundamental theorem of algebra.**

Is it true that any polynomial with coefficients from $\mathbb{Z}_M$ has at least one root in $\mathbb{Z}_M[i]$?

No, this is not true starting from 3rd degree. Just take polynomial $P(x) = x^r-a$ where a is not $r$-th residue. Example: $x^3+2$ for M=7.

**4. Solving cubic equation when M%4=1.** 

Do we really need the requirement $M \%3= 4$ to solve cubic equation? We only need to go into $Z_M[i]$ to compute square root, do an addition and compute cubic root, and computing roots doesn't involve divisions in $\mathbb{Z}_M[i]$. So it seems like exactly same code must work for all prime $M$s larger than 3, even if $Z_M[i]$ is not a field. Is this true?

**5. Computing discrete logarithm faster than $O(\sqrt{M})$.** 

Is it possible to compute discrete logarithm in an cyclic group of size $q$ in $O(poly(\log(q))$? If not, what is the lower bound? Is GSBS algorithm the fastest we can do?